In [3]:
# pip installs (run once per env)
# !pip install biopython requests py3Dmol


#https://www.uniprot.org/
#https://blast.ncbi.nlm.nih.gov/Blast.cgi
#https://swissmodel.expasy.org/

from Bio.Seq import Seq
import requests, textwrap, time
import py3Dmol


# 1. Protein Sequence of Human Lysozyme
protein_seq = Seq(
    "MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRAT"
    "NYNAGDRSTDYGIFQINSRYWCNDGKTPGAVNACHLSCSALLQDNIADAVACAKRVVRDP"
    "QGIRAWVAWRNRCQNRDVRQYVQGCGV"
)

if not protein_seq:
    raise ValueError("Translation produced an empty protein (check reading frame).")

print("Protein:", protein_seq)

# 2) Predict structure via ESMFold API (returns PDB as text)
api = "https://api.esmatlas.com/foldSequence/v1/pdb/"
resp = requests.post(api, data=protein_seq, timeout=120)

# handle rate limits or transient fails
tries = 0
while resp.status_code in (429, 503) and tries < 4:
    time.sleep(3 * (tries + 1))
    resp = requests.post(api, data=protein_seq, timeout=120)
    tries += 1

resp.raise_for_status()
pdb_data = resp.text

view = py3Dmol.view(width=700, height=500)
view.addModel(pdb_data, "pdb")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.addStyle({"hetflag": True}, {"stick": {}})
view.zoomTo()
view.show()

Protein: MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYNAGDRSTDYGIFQINSRYWCNDGKTPGAVNACHLSCSALLQDNIADAVACAKRVVRDPQGIRAWVAWRNRCQNRDVRQYVQGCGV


3Dmol.js failed to load for some reason. Please check your browser console for error messages.